In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# own code library
from config.config import *
from config.dataprep_config import *
from plotting import *
from dataprep.preprocessors import *

(config) no crisis measure selected.


In [2]:
abspath = r"C:\Users\Andy\PycharmProjects\finrlpaper2\MT-DRL-Pytorch"

# Preprocessing US stock dataset
Using intermediate dataset, where unfit company tics have already been removed.

In [3]:
df_us = pd.read_csv(os.path.join(abspath, "data", "intermediate", "US_stocks_WDB_a.csv"), index_col=0)
df_us.head()

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,prcld,prcod,prcstd,trfd,log_prccd
gvkey,,,,,,,,,,,,,,,
1690,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,37.875,NaN,3.0,1.083307,3.647406
1690,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,38.625,NaN,3.0,1.083307,3.673131
1690,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,38.750,NaN,3.0,1.083307,3.660351
1690,1,19950106,AAPL,APPLE INC,112.0,NaN,9613000.0,NaN,42.000,43.125,41.125,NaN,3.0,1.083307,3.737670
1690,1,19950109,AAPL,APPLE INC,112.0,NaN,2447000.0,NaN,41.203,41.875,41.000,NaN,3.0,1.083307,3.718511


#### Calculate: adjcp (adjusted clising price), open, high, low, volume

In [4]:
# function from preprocessors.py
df_us = calculate_price_volume_WhartonData(df=df_us.copy(), 
                                   new_cols_subset=data_settings.NEW_COLS_SUBSET, 
                                   target_subset=None)
df_us.head(3)

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,prcld,prcod,prcstd,trfd,log_prccd,adjcp,open,high,low,volume
0,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,37.875,NaN,3.0,1.083307,3.647406,0.342634,NaN,0.347098,0.338170,927400.0
1,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,38.625,NaN,3.0,1.083307,3.673131,0.351562,NaN,0.353795,0.344866,1416800.0
2,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,38.750,NaN,3.0,1.083307,3.660351,0.347098,NaN,0.351562,0.345982,657500.0


#### Calculate: technical indicators


In [5]:
# note: this usually takes up to 2 min
df_us = add_technical_indicator_with_StockStats(df=df_us, 
                                                technical_indicators_list=["macd", "rsi_30", "cci_30", "dx_30"])
df_us.head(3)

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,...,log_prccd,adjcp,open,high,low,volume,macd,rsi_30,cci_30,dx_30
0,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,...,3.647406,0.342634,NaN,0.347098,0.338170,927400.0,0.000000,NaN,NaN,NaN
1,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,...,3.673131,0.351562,NaN,0.353795,0.344866,1416800.0,0.000200,100.000000,66.666667,100.0
2,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,...,3.660351,0.347098,NaN,0.351562,0.345982,657500.0,0.000117,65.909091,28.571429,100.0


#### Calculate: other features, such as trading volume, volatility, return

In [6]:
df_us = add_other_features(df=df_us,
                       features=["returns_volatility", "return_daily", "log_return_daily"],
                       window_days_vola=7, # window is only relevant for volatility, as returns are always daily here
                       min_periods_vola=7, # min periods to be in window for calculatiom, otherwise NaN calculated
                       price_colum=data_settings.MAIN_PRICE_COLUMN,
                       asset_name_column=data_settings.ASSET_NAME_COLUMN)
df_us.rename(columns={"returns_volatility":"ret_vola_7d"}, inplace=True)

In [12]:
df_us["ret_vola_7d"].head(100)
df_us["ret_vola_7d"].isna().sum()/29 # for each of the 29 tickers

7.0

In [10]:
df_us = add_other_features(df=df_us,
                       features=["returns_volatility"],
                       window_days_vola=21, # 21 trading days in a month, usually
                       min_periods_vola=21,
                       price_colum=data_settings.MAIN_PRICE_COLUMN,
                       asset_name_column=data_settings.ASSET_NAME_COLUMN)
df_us.rename(columns={"returns_volatility":"ret_vola_21d"}, inplace=True)

In [13]:
df_us["ret_vola_21d"].head(100)
df_us["ret_vola_21d"].isna().sum()/29 

21.0

In [14]:
df_us = add_other_features(df=df_us,
                       features=["returns_volatility"],
                       window_days_vola=63, # 63 trading days in a quarter, usually
                       min_periods_vola=63,
                       price_colum=data_settings.MAIN_PRICE_COLUMN,
                       asset_name_column=data_settings.ASSET_NAME_COLUMN)
df_us.rename(columns={"returns_volatility":"ret_vola_63d"}, inplace=True)

In [15]:
df_us.head(5)

,iid,datadate,tic,conm,ajexdi,cshoc,cshtrd,eps,prccd,prchd,...,volume,macd,rsi_30,cci_30,dx_30,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
0,1,19950103,AAPL,APPLE INC,112.0,NaN,927400.0,NaN,38.375,38.875,...,927400.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,19950104,AAPL,APPLE INC,112.0,NaN,1416800.0,NaN,39.375,39.625,...,1416800.0,0.000200,100.000000,66.666667,100.000000,NaN,0.026059,0.025725,NaN,NaN
2,1,19950105,AAPL,APPLE INC,112.0,NaN,657500.0,NaN,38.875,39.375,...,657500.0,0.000117,65.909091,28.571429,100.000000,NaN,-0.012698,-0.012780,NaN,NaN
3,1,19950106,AAPL,APPLE INC,112.0,NaN,9613000.0,NaN,42.000,43.125,...,9613000.0,0.001083,89.360401,133.333333,100.000000,NaN,0.080386,0.077318,NaN,NaN
4,1,19950109,AAPL,APPLE INC,112.0,NaN,2447000.0,NaN,41.203,41.875,...,2447000.0,0.001303,75.633452,65.755057,94.113585,NaN,-0.018976,-0.019159,NaN,NaN


In [16]:
df_us.sort_values(by=["datadate", "tic"], inplace=True)
df_us["adjcp"].tail()

19974    NaN
39950    NaN
98779    NaN
172023   NaN
33291    NaN
Name: adjcp, dtype: float64

In [17]:
df_us.loc[df_us["datadate"]>=20210611, ["datadate", "tic", "adjcp"]]
### TODO: remove Nan, especially after date 14.6.2021: df[df["datadate"]>=20210614] (last load was on 13.6.2021)

,datadate,tic,adjcp
6657,20210611,AAPL,127.35
13315,20210611,AMGN,242.77
19973,20210611,AXP,164.51
26632,20210611,BA,247.28
33290,20210611,CAT,220.70
39949,20210611,CSCO,54.77
46608,20210611,CVX,107.91
53266,20210611,DIS,177.38
58830,20210611,GS,378.05
65488,20210611,HD,310.77


In [18]:
# removing data that goes beyond the latest fetching date (20210611)
df_us = df_us[df_us["datadate"]<=20210611]

In [19]:
df_us["adjcp"].tail()

165363    397.89
172022     57.33
178681     55.31
185339    140.75
191997     62.17
Name: adjcp, dtype: float64

In [20]:
df_us.columns

Index(['iid', 'datadate', 'tic', 'conm', 'ajexdi', 'cshoc', 'cshtrd', 'eps',
       'prccd', 'prchd', 'prcld', 'prcod', 'prcstd', 'trfd', 'log_prccd',
       'adjcp', 'open', 'high', 'low', 'volume', 'macd', 'rsi_30', 'cci_30',
       'dx_30', 'ret_vola_7d', 'return_daily', 'log_return_daily',
       'ret_vola_21d', 'ret_vola_63d'],
      dtype='object')

In [21]:
relevant_cols = ['datadate', 'tic','eps','adjcp', 'open', 'high', 'low', 'volume', 'macd', 'rsi_30', 'cci_30',
                 'dx_30', 'ret_vola_7d', 'return_daily', 'log_return_daily','ret_vola_21d', 'ret_vola_63d']

In [22]:
df_us = df_us[relevant_cols]
df_us.head()

,datadate,tic,eps,adjcp,open,high,low,volume,macd,rsi_30,cci_30,dx_30,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
0,19950103,AAPL,NaN,0.342634,NaN,0.347098,0.338170,927400.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6658,19950103,AMGN,NaN,7.281250,NaN,7.390625,7.250000,938700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13316,19950103,AXP,NaN,9.750000,NaN,9.833333,9.666667,1321700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19975,19950103,BA,NaN,23.375000,NaN,23.562500,23.062500,649600.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26633,19950103,CAT,NaN,13.750000,NaN,13.781250,13.656250,531100.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
print(f"dataset length: {len(df_us)}")
df_us[df_us.datadate>=20000101].isna().sum() #eps and open have many missing values, even from 2000 on
# hence we drop them

dataset length: 191990


datadate                0
tic                     0
eps                   307
adjcp                   2
open                33874
high                    2
low                     2
volume                  2
macd                    0
rsi_30                  0
cci_30                  2
dx_30                   0
ret_vola_7d             0
return_daily            0
log_return_daily        4
ret_vola_21d            0
ret_vola_63d            0
dtype: int64

In [ ]:
df_us.drop(columns=["eps", "open"], inplace=True)

In [38]:
########TODO
df_us = df_us[df_us.datadate>=20000101]

In [40]:
df_us[df_us.datadate>=20000101].isna().sum()

datadate            0
tic                 0
adjcp               2
high                2
low                 2
volume              2
macd                0
rsi_30              0
cci_30              2
dx_30               0
ret_vola_7d         0
return_daily        0
log_return_daily    4
ret_vola_21d        0
ret_vola_63d        0
dtype: int64

In [41]:
df_us[df_us['adjcp'].isna()] # can look up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_30,cci_30,dx_30,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,59.325473,NaN,42.889909,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,53.644404,NaN,1.301765,0.015348,0.0,NaN,0.014570,0.020281


In [42]:
df_us[df_us["high"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_30,cci_30,dx_30,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,59.325473,NaN,42.889909,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,53.644404,NaN,1.301765,0.015348,0.0,NaN,0.014570,0.020281


In [43]:
df_us[df_us["low"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_30,cci_30,dx_30,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,59.325473,NaN,42.889909,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,53.644404,NaN,1.301765,0.015348,0.0,NaN,0.014570,0.020281


In [44]:
df_us[df_us["volume"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_30,cci_30,dx_30,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,59.325473,NaN,42.889909,0.012710,0.0,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,53.644404,NaN,1.301765,0.015348,0.0,NaN,0.014570,0.020281


In [45]:
df_us[df_us["log_return_daily"].isna()] # can lok up in another data bank or impute, since only one value per ticker missing
# stock: KO, NKE

,datadate,tic,adjcp,high,low,volume,macd,rsi_30,cci_30,dx_30,ret_vola_7d,return_daily,log_return_daily,ret_vola_21d,ret_vola_63d
100469,20010912,KO,NaN,NaN,NaN,NaN,0.505423,59.325473,NaN,42.889909,0.012710,0.000000,NaN,0.013867,0.013617
133762,20010913,NKE,NaN,NaN,NaN,NaN,0.069560,53.644404,NaN,1.301765,0.015348,0.000000,NaN,0.014570,0.020281
100470,20010917,KO,25.10000,25.1000,24.35500,10128300.0,0.520251,59.325473,88.806578,42.889909,0.012444,0.005005,NaN,0.013863,0.013619
133763,20010917,NKE,5.50625,5.6875,5.49625,2371700.0,-0.003167,53.644404,-318.351364,1.301765,0.036960,-0.094553,NaN,0.025059,0.023625


In [48]:
### for now, I SIMPLY DROP KO AND NKE
df_us = df_us[~df_us["tic"].isin(["KO", "NKE"])]
df_us.tic.unique()

array(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS',
       'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MMM', 'MRK',
       'MSFT', 'PFE', 'PG', 'RTX', 'UNH', 'VZ', 'WBA', 'WMT', 'XOM'],
      dtype=object)

In [49]:
df_us[df_us.datadate>=20000101].isna().sum() # no missing values anymore, hence we can save it

datadate            0
tic                 0
adjcp               0
high                0
low                 0
volume              0
macd                0
rsi_30              0
cci_30              0
dx_30               0
ret_vola_7d         0
return_daily        0
log_return_daily    0
ret_vola_21d        0
ret_vola_63d        0
dtype: int64

In [50]:
df_us.to_csv(os.path.join(abspath, "data", "preprocessed", "US_stocks_WDB_try.csv"))

### Market Indicators

In [ ]:
df_us[df_us.datadate>=2000].head(100) # drop eps, open 